# Example 02 - GRNBoost2 with custom Dask `Client`

In this example notebook, we illustrate a slightly more advanced scenario where we
infer the gene regulatory network from a single dataset, using a custom Dask client.

In [1]:
import os
import pandas as pd

from arboreto.algo import grnboost2, genie3
from arboreto.utils import load_tf_names

## 1. Load the input data

* We use the [Pandas](http://pandas.pydata.org/) library to read the data from a tab-separated text file.
* Arboreto expects the `expression_data` matrix to have observations as rows and genes as columns.

In [2]:
wd = os.getcwd().split('arboreto')[0] + 'arboreto/resources/dream5/'

net1_ex_path = wd + 'net1/net1_expression_data.tsv'
net1_tf_path = wd + 'net1/net1_transcription_factors.tsv'

In [3]:
ex_matrix = pd.read_csv(net1_ex_path, sep='\t')

* Let's quickly check the the input matrix by inspecting its shape and top 5 rows.

In [4]:
ex_matrix.shape

(805, 1643)

In [5]:
ex_matrix.head()

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,...,G1634,G1635,G1636,G1637,G1638,G1639,G1640,G1641,G1642,G1643
0,0.425448,0.017829,0.907989,0.448247,0.172324,0.273489,0.843766,0.648201,1.004533,0.365305,...,0.011979,0.963306,1.169870,0.331381,0.350600,0.822844,0.304483,0.319917,0.364280,0.765945
1,0.442400,0.050525,0.869368,0.445851,0.173311,0.274889,0.764049,0.747870,1.022589,0.434106,...,0.022247,1.014137,0.888465,0.281649,0.485940,0.915617,0.317507,0.238074,0.509130,0.691403
2,1.056847,0.208454,0.467448,0.505077,0.244883,0.208451,0.665355,1.192092,0.824068,0.146987,...,0.422066,0.895203,1.028826,0.825126,0.444819,0.349069,0.042310,0.165208,0.952178,0.678781
3,1.117226,0.003001,0.317654,0.387204,0.253792,0.179360,0.939244,0.868668,0.963028,0.233785,...,0.001163,1.046540,1.058098,0.484225,0.150689,0.449126,0.125197,0.000047,0.878127,0.566691
4,0.971068,0.001056,0.354651,0.474532,0.207718,0.102833,0.745871,0.909753,1.151865,0.318988,...,0.000845,1.041745,1.061129,0.384363,0.326859,0.512270,0.261410,0.000156,0.883981,0.646715


* We load the transcription factor (TF) list from a file using the `load_tf_names` utility function which simply reads a file line per line where every line contains one TF name.

In [6]:
tf_names = load_tf_names(net1_tf_path)

* Some quick inspections

In [7]:
tf_names[:5]

['G1', 'G2', 'G3', 'G4', 'G5']

In [8]:
len(tf_names)

195

## 2. Create a custom Dask `Client`

In this section, we create a custom `Client`, connected to a customly configured Dask.distributed `LocalCluster` instance.

In this case, we specify the number of worker processes we wish to spin up 

> Pick a number that is smaller than the number of CPU threads on your machine. A laptop typically has 8, a high end Intel Xeon workstation can have tens of hyperthreaded cores.

Notice that the `Client` now features a dashboard url. This takes us to the Dask distributed [web interface](http://distributed.readthedocs.io/en/latest/web.html) where we can monitor the progress of our inference job.

In [9]:
from distributed import LocalCluster, Client

In [10]:
local_cluster = LocalCluster(n_workers=10, 
                             threads_per_worker=1)

custom_client = Client(local_cluster)

In [11]:
custom_client

Client Scheduler: tcp://127.0.0.1:45199 Dashboard: http://127.0.0.1:8787,Cluster Workers: 10 Cores: 10 Memory: 15.35 GB


If your notebook is running on a remote server, you can use port forwarding to access the dashboard on your local machine.

```bash
$ ssh -L 9999:localhost:8787 <remote server>
```

> The right port `8787` is the server port on which the dashboard server is running, the left port `9999` is the port on which to view the dashboard locally on url: 
`localhost:9999`

## 3. Launch gene regulatory network inference

In [12]:
%%time
network = grnboost2(expression_data=ex_matrix,
                    tf_names=tf_names,
                    client_or_address=custom_client)

CPU times: user 46.3 s, sys: 11.9 s, total: 58.2 s
Wall time: 2min 56s


In [13]:
network.head()

,TF,target,importance
15,G16,G687,140.035988
108,G109,G1406,133.392335
15,G16,G1440,127.383061
187,G188,G938,125.987409
61,G62,G360,114.653689


In [14]:
len(network)

318731

## 4. Write the GRN link list to file `[TF, target, importance]`.

In [18]:
network.to_csv('ex_02_network.tsv', sep='\t', header=False, index=False)

## 5. Close the `Client` and `LocalCluster`

In [17]:
custom_client.close()
local_cluster.close()